In [1]:
from __future__ import print_function
import numpy as np
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse.linalg import svds
from sklearn.preprocessing import normalize

In [2]:
#opens json file and creates a list of tuples with name of video and description called documents
#data represents a list of dictionarys for each video with all of the info from ted_main

In [3]:
documents=[]
with open("ted_main.json", encoding="utf8") as f:
    data=json.load(f)
    for x in data:
        documents.append((x["name"], x["description"]))

In [7]:
#given the data file and the title of the video, outputs the index that video is located at

In [8]:
def findindex(data,title):
    for x in range(len(data)):
        if data[x]['name']== title:
            return x

In [11]:
#creates tf-idf matrix, can alter max_df and min_df

In [70]:
vectorizer = TfidfVectorizer(stop_words = 'english', max_df = .75,
                            min_df = 50)
my_matrix = vectorizer.fit_transform([x[1] for x in documents]).transpose()

In [71]:
#runs svd, can alter k but values mostly live in the space under 30

In [77]:
words_compressed, _, docs_compressed = svds(my_matrix, k=40)
docs_compressed = docs_compressed.transpose()
print(docs_compressed.shape)

(2550, 40)


In [78]:
#creates clusters of 15 vidoes based on the index of the input video

In [79]:
docs_compressed = normalize(docs_compressed, axis = 1)
def closest_projects(project_index_in, docs_compressed, k = 15):
    sims = docs_compressed.dot(docs_compressed[project_index_in,:])
    asort = np.argsort(-sims)[:k+1]
    return [(documents[i][0],sims[i]/sims[asort[0]]) for i in asort[1:]]

In [80]:
print(data[0]['title'])
print(closest_projects(0, docs_compressed))

Do schools kill creativity?
[('Jonas Gahr Støre: In defense of dialogue', 0.9760434138352786), ('Lauren Zalaznick: The conscience of television', 0.9084247925540214), ('Kim Gorgens: Protecting the brain against concussion', 0.8765008966286858), ('Myriam Sidibe: The simple power of hand-washing', 0.8667751607420577), ('Noah Feldman: Politics and religion are technologies', 0.8607179924057017), ('Marcel Dicke: Why not eat insects?', 0.8586061011086409), ('Jeff Bezos: What matters more than your talents', 0.8493471009675864), ('Richard Dawkins: Why the universe seems so strange', 0.8254184830572127), ('Peter Diamandis: Abundance is our future', 0.8245982141771252), ('Simon Sinek: Why good leaders make you feel safe', 0.8155052269849342), ('Malcolm Gladwell: Choice, happiness and spaghetti sauce', 0.8122878840159172), ('Gary Vaynerchuk: Do what you love (no excuses!)', 0.8104308009422787), ('Shimon Steinberg: Natural pest control ... using bugs!', 0.8064450739975075), ('Rory Sutherland: Pe

In [82]:
print(data[1]['title'])
print(closest_projects(1, docs_compressed))

Averting the climate crisis
[('Juan Enriquez: Using biology to rethink the energy challenge', 0.7828148297970142), ('Audrey Choi: How to make a profit while making a difference', 0.7454212282344344), ('Jill Sobule: Global warming\'s theme song, "Manhattan in January"', 0.730664606773714), ('Becci Manson: (Re)touching lives through photos', 0.730664606773714), ('Hugh Evans: What does it mean to be a citizen of the world?', 0.7254080005817897), ('The LXD: In the Internet age, dance evolves ...', 0.7159747720858395), ('Al Gore: New thinking on the climate crisis', 0.7128493025584265), ('Charmian Gooch: My wish: To launch a new era of openness in business', 0.6669067057312604), ('Drew Berry: Animations of unseeable biology', 0.6542698563618953), ('Bjorn Lomborg: Global priorities bigger than climate change', 0.6428088359054435), ('Paddy Ashdown: The global power shift', 0.6351366455694082), ('Michael Green: How we can make the world a better place by 2030', 0.6330525223310709), ('Stefano M

In [84]:
print(data[3]['title'])
print(closest_projects(3, docs_compressed))

Greening the ghetto
[('Bjarke Ingels: Hedonistic sustainability', 0.9621206454060154), ('Jonathan Drori: The beautiful tricks of flowers', 0.8576943260899125), ('Richard Resnick: Welcome to the genomic revolution', 0.8563805830890029), ('Danielle de Niese: A flirtatious aria', 0.8093641507997104), ('Seth Priebatsch: The game layer on top of the world', 0.8082739617536385), ('Roger Doiron: My subversive (garden) plot', 0.7893934646897841), ('Jeff Han: The radical promise of the multi-touch interface', 0.7573038840658094), ('Cynthia Schneider: The surprising spread of Idol TV', 0.7473486211261559), ('Carolyn Porco: This is Saturn', 0.7427640790168326), ('Kate Hartman: The art of wearable communication', 0.740725285833474), ('Shereen El Feki: Pop culture in the Arab world', 0.7358841876638001), ('Kamal Meattle: How to grow fresh air', 0.7296672931726582), ('Svante Pääbo: DNA clues to our inner neanderthal', 0.718670625355767), ("Richard Pyle: A dive into the reef's Twilight Zone", 0.70944

In [33]:
for i in range(10):
    print(documents[i][0])
    for title, score in closest_projects(i):
        print("{}:{:.3f}".format(title[:40], score))
    print()

Ken Robinson: Do schools kill creativity?


TypeError: closest_projects() missing 1 required positional argument: 'docs_compressed'

In [34]:
#function that takes in the closest_projects function and the index of the desirable video 
#outputs the url of the second video in the cluster

In [20]:
def extract_cluster_ratings(index):
    lst=closest_projects(index)
    print(lst)
    name=lst[1][0]
    print(name)
    index=findindex(data,name)
    print(index)
    return data[index]["url"]

In [22]:
import ast

In [35]:
def moodvid(lst,mood,data):
    mx=0
    vid=0;
    for x in range(len(lst)):
        name=lst[x][0]
        index=findindex(data,name)
        ratings=ast.literal_eval(data[index]['ratings'])
        for el in range(len(ratings)):
            if ratings[el]['name']==mood:
                count= ratings[el]['count']
                if count > mx:
                    mx=count
                    vid=index
    return vid
    

In [85]:
def moodvid(lst,mood,data): #returning 1 video
    mx=0
    vid=0;
    vid_mood_lst = []
    for x in range(len(lst)):
        name=lst[x][0]
        index=findindex(data,name)
        ratings=ast.literal_eval(data[index]['ratings'])
        for el in range(len(ratings)):
            if ratings[el]['name']==mood:
                count= ratings[el]['count']
                vid_mood_lst.append((index,count))
        sorted_vid_mood_lst = sorted(vid_mood_lst, key=lambda tup: tup[1])
    return sorted_vid_mood_lst

In [25]:
def extract_cluster_ratings2(index, mood):
    lst=closest_projects(index)
    i=moodvid(lst, mood, data)
    return data[i]["url"]

In [86]:
def top_svd(data, index, mood):
    print(data[index]['name'])
    lst=closest_projects(index, docs_compressed)
    ranked_vids = moodvid(lst,mood,data) #tuple of index and count
    print(ranked_vids)
    top_vids = []
    for i in range(10):
        ind = ranked_vids[i][0]
        temp_url = data[ind]['url']
        print(data[ind]['title'])
        print(data[ind]['description'])
        embed_url = "https://embed.ted.com/talks/" + temp_url[26:]
        top_vids.append([embed_url, i-1, data[ind]['title'], data[ind]['description']])
    return top_vids


In [87]:
closest_projects(0, docs_compressed)

[('Jonas Gahr Støre: In defense of dialogue', 0.9760434138352786),
 ('Lauren Zalaznick: The conscience of television', 0.9084247925540214),
 ('Kim Gorgens: Protecting the brain against concussion', 0.8765008966286858),
 ('Myriam Sidibe: The simple power of hand-washing', 0.8667751607420577),
 ('Noah Feldman: Politics and religion are technologies', 0.8607179924057017),
 ('Marcel Dicke: Why not eat insects?', 0.8586061011086409),
 ('Jeff Bezos: What matters more than your talents', 0.8493471009675864),
 ('Richard Dawkins: Why the universe seems so strange', 0.8254184830572127),
 ('Peter Diamandis: Abundance is our future', 0.8245982141771252),
 ('Simon Sinek: Why good leaders make you feel safe', 0.8155052269849342),
 ('Malcolm Gladwell: Choice, happiness and spaghetti sauce',
  0.8122878840159172),
 ('Gary Vaynerchuk: Do what you love (no excuses!)', 0.8104308009422787),
 ('Shimon Steinberg: Natural pest control ... using bugs!',
  0.8064450739975075),
 ('Rory Sutherland: Perspective i

In [90]:
top_svd(data, 1, "Inspiring")

Al Gore: Averting the climate crisis
[(86, 25), (2146, 156), (1703, 200), (1109, 224), (161, 246), (51, 280), (1305, 294), (786, 300), (1166, 302), (1102, 311), (2063, 413), (631, 543), (491, 711), (2181, 859), (214, 1435)]
Global warming's theme song, "Manhattan in January"
A happy song about global warming, from Jill Sobule.
How to make a profit while making a difference
Can global capital markets become catalysts for social change? According to investment expert Audrey Choi, individuals own almost half of all global capital, giving them (us!) the power to make a difference by investing in companies that champion social values and sustainability. "We have more opportunity today than ever before to make choices," she says. "So change your perspective. Invest in the change you want to see in the world."
My wish: To launch a new era of openness in business
Anonymous companies protect corrupt individuals – from notorious drug cartel leaders to nefarious arms dealers – behind a shroud of 

[['https://embed.ted.com/talks/jill_sobule_sings_to_al_gore\n',
  -1,
  'Global warming\'s theme song, "Manhattan in January"',
  'A happy song about global warming, from Jill Sobule.'],
 ['https://embed.ted.com/talks/audrey_choi_how_to_make_a_profit_while_making_a_difference\n',
  0,
  'How to make a profit while making a difference',
  'Can global capital markets become catalysts for social change? According to investment expert Audrey Choi, individuals own almost half of all global capital, giving them (us!) the power to make a difference by investing in companies that champion social values and sustainability. "We have more opportunity today than ever before to make choices," she says. "So change your perspective. Invest in the change you want to see in the world."'],
 ['https://embed.ted.com/talks/charmian_gooch_my_wish_to_launch_a_new_era_of_openness_in_business\n',
  1,
  'My wish: To launch a new era of openness in business',
  'Anonymous companies protect corrupt individuals –